In [29]:
from yggdrasil.examples import yamls as ex_yamls
import yaml
import jupyterlab_nodeeditor as jlne

In [30]:
# Reworked version of making a JLNE-compliant dictionary from a Yggdrasil Model YAML
# Works with the defined Trifecta YAML
# We want each model to be it's own component
# This can be changed later

def dict_conversion(model_file):
    # Using flags here in case I need them later, will remove if I don't
    is_dict, is_list = False, False
    
    # There is no clear indicator if a safeloaded YAML will have single or multiple models
    # Should parse them as I go, but having an indicator for it being a list/dict is still important
    
    # If it's neither, return the type so I can work on it
    if type(model_file) is dict:
        is_dict = True
    elif type(model_file) is list:
        is_list = True
    else:
        return ("Unaccounted model type: ", type(model_file))
    
    # I'll leave this intialization in since it can be worked around
    # The main problem is multiple models in a dictionary, so let's have it first check for multiple models
    if is_dict:       
        # Simplest way to check for multiple models
        list_of_models = []
        
        if "models" in model_file.keys():
            for model in model_file["models"]:
                # Setup dictionary to be filled
                new_dict, new_dict["inputs"], new_dict["outputs"], new_dict['title'] = {}, [], [], model['name']
                
                # Fill in the Inputs
                if "input" in model.keys():
                    for i, inp in enumerate(model["input"]):
                        new_dict["inputs"].append({'title': inp["name"], 'key': f"temp_in{i}", 'socket_type': "connection"})
                elif "inputs" in model.keys():
                    for i, inp in enumerate(model["inputs"]):
                        new_dict["inputs"].append({'title': inp["name"], 'key': f"temp_in{i}", 'socket_type': "connection"})
        
                # Fill in the Outputs, same as inputs with name changes
                if "output" in model.keys():
                    for o, out in enumerate(model["output"]):
                        new_dict["outputs"].append({'title': out["name"], 'key': f"temp_out{o}", 'socket_type': "connection"})
                elif "outputs" in model.keys():
                    for o, out in enumerate(model["outputs"]):
                        new_dict["outputs"].append({'title': out["name"], 'key': f"temp_out{o}", 'socket_type': "connection"})
                list_of_models.append(new_dict)
        
        # We return a list here so that way loading the models is uniform for both multi-model and single-model files
        return list_of_models
    

    # ENTIRE LIST SECTION IS TO BE REFACTORED
    
    # We can pretty much do the dictionary loop multiple times per model then return the list with minor adjustments noted below
#     elif is_list:
#         model_list = []
#         for model in model_file:
#             # Setup initial dictionary to be filled
#             new_dict, new_dict["inputs"], new_dict["outputs"],  = {}, [], []
#             if "name" in model.keys():
#                 new_dict["title"] = model_file["name"]
#             else:
#                 new_dict["title"] = model_name
        
#             # Fill in the Inputs, no enumeration needed here
#             if "input" in model.keys():
#                 if "type" in model["input"][0]:
#                     new_dict["inputs"].append({'title': model["input"][0]["name"], 'key': "temp_in", 'socket_type': model["input"][0]["type"]})
#                 else:
#                     new_dict["inputs"].append({'title': model["input"], 'key': "temp_in", 'socket_type': "temp_socket"})
#             elif "inputs" in model.keys():
#                 if "type" in model["inputs"][0]:
#                     new_dict["inputs"].append({'title': model["inputs"][0]["name"], 'key': "temp_in", 'socket_type': model["inputs"][0]["type"]})
#                 else:
#                     new_dict["inputs"].append({'title': model["inputs"], 'key': "temp_in", 'socket_type': "temp_socket"})
        
#             # Fill in the Outputs, same as inputs with name changes
#             if "output" in model.keys():
#                 if "type" in model["output"][0]:
#                     new_dict["outputs"].append({'title': model["output"][0]["name"], 'key': "temp_in", 'socket_type': model["output"][0]["type"]})
#                 else:
#                     new_dict["outputs"].append({'title': model["output"], 'key': "temp_in", 'socket_type': "temp_socket"})
#             elif "outputs" in model.keys():
#                 if "type" in model["outputs"][0]:
#                     new_dict["outputs"].append({'title': model["outputs"][0]["name"], 'key': "temp_in", 'socket_type': model["outputs"][0]["type"]})
#                 else:
#                     new_dict["outputs"].append({'title': model["outputs"], 'key': "temp_in", 'socket_type': "temp_socket"})
                    
#             model_list.append(new_dict)
        
#         # Return the full converted list of models
#         return model_list

In [31]:
# ps - Node Editor instance that it is added to, default blank
def load_model(ps = None):
    # E.g. filepath: sample_models/sample_1.yml
    # Photosynthesis broken right now with tentative changes
    filepath = input("Please enter filepath for model: ")
    
    # Variable that will be used as the processed model
    fsample = None
    
    # Check to see if input is blank or not, run the respective code to get the model from file or the Photosynthesis example
    if filepath:
        with open(filepath, "r") as sample:
            fsample = yaml.safe_load(sample)
            
    else:
        print("Loading Photosynthesis Model into Editor...")
        with open(ex_yamls['fakeplant']['python'], "r") as sample:
            fsample = yaml.safe_load(sample)["model"]
    
    # Need to automate the jlne portion due to some bugs
    
    # This is our socket collection: a list that is converted to a tuple at each instance of adding a component
    socket_list = []
    
    # Convert it all and add it into the editor, then return the entire editor
    ne_instance = ps or jlne.NodeEditor()
    model_list = dict_conversion(fsample)
    for model in model_list:
        # print(model)
        model_ins, model_outs = [], []
        # Might need to adjust if it's model["input"] vs model["inputs"] but rushing to finish here
        # Can also enumerate for unique keys if needed (overwrites the keys in the parser)
        for model_in in model["inputs"]:
            # print("model_in: ", model_in)
            socket_list.append(model_in["socket_type"])
            model_ins.append(jlne.InputSlot(title = model_in["title"], key = model_in["key"], socket_type = model_in["socket_type"], sockets = jlne.SocketCollection(socket_types = tuple(socket_list))))
            
        for model_out in model["outputs"]:
            # print("model_out: ", model_out)
            socket_list.append(model_out["socket_type"])
            model_outs.append(jlne.OutputSlot(title = model_out["title"], key = model_out["key"], socket_type = model_out["socket_type"], sockets = jlne.SocketCollection(socket_types = tuple(socket_list))))
        
        ne_instance.add_component(jlne.Component(sockets = jlne.SocketCollection(socket_types = tuple(socket_list)), inputs = model_ins, outputs = model_outs, title = model["title"]))
    
    return ne_instance